# Ase calculator qe

To add the QE implmentation as a model to the test-framework, the values given to the qe calculator cannot depend on the spcific calculation being run or the atoms object. One solution to this is by including any atom dependent configs into the calculator itself. Ie for the calculator to adjust parameters based on the atoms object its calculating directly. 

The idea is hence to modify the calculator, such that for the slab systems the kpoints are adjusted inside the calculator function. 

There are a few QE parameters that currently depend on the specific atoms object which is being calculated. These are 

1. Surfaces
    - stress, kpoints, dipole
    ```python
    calcstress = False
    print("type {}: dipole correction on".format(info["type"]))
    dipole = {"status": True}
    kpts[2] = 1
    ```
2. Isolated Atoms
    - stress, spinpol, "initial_magmoms"
    ```python
    calcstress = False
    spinpol = False
    kpts = "gamma"
    atoms.arrays.pop("initial_magmoms")
    ```
3. General
    - kpoints
    ```python
    kspacing = 0.25 / (2.0 * np.pi)
    kpts = kspacing_to_grid(atoms, spacing=kspacing)
    ```
    - prefix `prefix = "QE_{}_0".format(info["uid"])`
   

    
    
Proposed fixing: 
- We add a variable Espresso( ..., slab_config={}), where {} is a dictionary of variables which will overwriten if the configuration is a slab configuration
    

In [6]:
import ase.io
import numpy as np
ats = ase.io.read('/data/lls34/Data/In2O3/In2O3_LS_1/04_QE2/03_Observalbes/01_Surface_Energy/non-symmetric/Original_Surfaces/Ia3_8_8.xyz')

atoms = ats.copy()
ats.center(vacuum=0)
diff = atoms.cell-ats.cell
np.arange(3)[np.sum(diff>1, axis=1) != 0]

array([2])

In [7]:
np.linalg.norm([[1,2,0],[2,0,0], [1,0,0]], axis=1)

array([2.23606798, 2.        , 1.        ])

In [8]:
a = [0,1,2,3]

In [9]:
b = [1,3]
for i in b:
    a[i] = 10
a

[0, 10, 2, 10]

In [10]:
np.max(np.linalg.norm(atoms.cell,axis=1))

84.93236404719576

In [11]:
atoms.get_volume()

37147.97852470845

# Just QE calculator

In [12]:
from espresso.espresso import Espresso            # Espresso Library
from ase import units # Ase Units
from espresso.siteconfig import SiteConfig
from ase.io import read, write
from ase.io.espresso import kspacing_to_grid
import sys
import os
import numpy as np

In [15]:
"""
Python script for running QE with the github calculator

This was originally created by Sandip on Feb 2021. The code requires the espresso
.esspresso to be installed. This was modified to work on womble0 using the GridEngine

The script can be called by

`python qe_simple.py traj.xyz 2`

where 2 is the configuraiton in traj.xyz that should be calculated

 - still need to do out directory
"""

from espresso.espresso import Espresso            # Espresso Library
from ase import units # Ase Units
from espresso.siteconfig import SiteConfig
from ase.io import read, write
from ase.io.espresso import kspacing_to_grid
import sys
import os
import numpy as np


# ---- Setting Variables

# Environements
os.environ["SCRATCH"] = "/home/lls34/trash/scratch/"

PSP_PATH = (
    "/home/lls34/programs/QuantumExpresso/pseudopotentials/use_pot"  # Pseudo Potentails
)

# Sceduling Properties
#site = SiteConfig("GE", scratchenv="SCRATCH")
#site = SiteConfig(scheduler=None)



# Atom specific properties

atoms = atoms

info = {"uid": atoms.info["uid"], "type": atoms.info["type"]}

prefix = "QE_{}_0".format(info["uid"])

j = 0
# Make sure unique output file
while os.path.exists(os.path.join(prefix)) or os.path.exists(
    os.path.join(prefix, ".xyz")
):
    prefix = "QE_{}_{}".format(info["uid"], j)
    j += 1

    if j > 1000:
        raise ValueError("While loop ran too often")
print(f"New prefix {prefix}")

# Now Properites


kspacing = 0.25 / (2.0 * np.pi)

kpts = kspacing_to_grid(atoms, spacing=0.25 / (2.0 * np.pi))
xc = "PBE"
dipole = {"status": False}
calcstress = True
spinpol = True

convergence = {
    "energy": 1e-7,
    "mixing": 0.5,
    "nmix": 10,
    "maxsteps": 500,
    "diag": "david",
}

if "Slab" in info["type"]:
    calcstress = False
    print("type {}: dipole correction on".format(info["type"]))
    dipole = {"status": True}
    kpts[2] = 1

print(f"using kpts: {kpts}")

magmom_dict = {"In": 10, "O": 4, "C": 3, "H": 1, "N": 3}
magmoms = []

for i in range(len(atoms)):
    magmoms.append(magmom_dict[atoms[i].symbol])

atoms.set_initial_magnetic_moments(magmoms)

if "isolated_atom" in info["type"]:
    calcstress = False
    spinpol = False
    kpts = "gamma"
    atoms.arrays.pop("initial_magmoms")

calc = Espresso(
    # pseudopotentials=pseudopotentials,
    pw=65 * units.Ry,  # plane-wave cut-off in eV, defaults to 350.0
    dw=8 * 65 * units.Ry,  # default 10*pw
    xc=xc,
    kpts=kpts,
    calculation="scf",
    psppath=PSP_PATH,
    parflags="-npool 1",
    nbands=-20,
    occupations="smearing",
    sigma=0.1,
    smearing="gaussian",
    convergence=convergence,
    dipole=dipole,
    calcstress=calcstress,
    spinpol=spinpol,
    # vdw_corr='D3',
    output={
        "removesave": False,
        "avoidio": False,
        "removewf": False,
        "wf_collect": True,
    },
    outdir=prefix,
    site=None,#site,
    txt="pw.out",
)



New prefix QE_599afbca03b14e7bbd6a9f7eb00e5c09_0
using kpts: [1, 3, 1]


In [16]:
try:
    atoms.set_calculator(calc)
    print(f"PE: {atoms.get_potential_energy()}")
    atoms.info = info
    write("{}.xyz".format(prefix), [atoms], format="extxyz")
    atoms = read("{}.xyz".format(prefix))
    info = atoms.info
    if "stress" in atoms.info.keys():
        info["virial"] = -atoms.info["stress"] / atoms.get_volume()
        atoms.info = info
        print("writing new")
        write("{}.xyz".format(prefix), [atoms], format="extxyz")

except ImportError as e:
    open("{}.FAIL".format(prefix), "w").close()
    print(f"Error: {prefix}, {e}")


check type config
Dipol correction on
kpts[[2]] = 1
2
['             z_valence', '"1.300000000000000E+001"\n']
2
['             z_valence', '"6.000000000000000E+000"\n']
2
['             z_valence', '"1.300000000000000E+001"\n']
2
['             z_valence', '"6.000000000000000E+000"\n']
2 (array([13, 13, 13, ...,  6,  6,  6]), {'In': 13, 'O': 6})
2
['             z_valence', '"1.300000000000000E+001"\n']
2
['             z_valence', '"6.000000000000000E+000"\n']


KeyboardInterrupt: 

## Writing to json file

In [30]:
import json


NameError: name '__file__' is not defined

In [31]:
# Environements
os.environ["SCRATCH"] = "/home/lls34/trash/scratch/"

PSP_PATH = (
    "/home/lls34/programs/QuantumExpresso/pseudopotentials/use_pot"  # Pseudo Potentails
)

config = {
    'SCRATCH': "/home/lls34/trash/scratch/",
    'PSP_PATH': '/home/lls34/programs/QuantumExpresso/pseudopotentials/use_pot',
    'test':None
}

with open('config.yml', 'w') as outfile:
    json.dump(config, outfile)

In [32]:
json.load(open('config.yml'))

{'SCRATCH': '/home/lls34/trash/scratch/',
 'PSP_PATH': '/home/lls34/programs/QuantumExpresso/pseudopotentials/use_pot',
 'test': None}

In [33]:
!cat config.yml

{"SCRATCH": "/home/lls34/trash/scratch/", "PSP_PATH": "/home/lls34/programs/QuantumExpresso/pseudopotentials/use_pot", "test": null}

In [ ]:
def read_data(model_names):
    data = {}
    for model_name in model_names:
        json_file = f"Si-model-{model_name}-test-bulk_Si_diamond-properties.json"
        model_name = os.path.basename(json_file).split('-')[2]
        data[model_name] = json.load(open(json_file))
    return data
    
data = read_data(['CASTEP_ASE', 'GAP'])
ref_model_name = 'CASTEP_ASE'

# With new ase-espresso

In [30]:
pwd

'/home/lls34/GitHub/01_PhD/PhD_Code/submodules/testing-framework/testing-framework/examples'

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [29]:
"""
Python script for running QE with the github calculator

This was originally created by Sandip on Feb 2021. The code requires the espresso
.esspresso to be installed. This was modified to work on womble0 using the GridEngine

The script can be called by

`python qe_simple.py traj.xyz 2`

where 2 is the configuraiton in traj.xyz that should be calculated

 - still need to do out directory
"""

from espresso.espresso import Espresso            # Espresso Library
from ase import units # Ase Units
from espresso.siteconfig import SiteConfig
from ase.io import read, write
from ase.io.espresso import kspacing_to_grid
import sys
import os
import numpy as np


# ---- Setting Variables

# Environements
os.environ["SCRATCH"] = "/home/lls34/trash/scratch/"

atoms = ase.io.read('/home/lls34/GitHub/01_PhD/PhD_Code/submodules/testing-framework/testing-framework/examples/In_mp-1055994_primitive.cif')
os.environ['SGE_O_WORKDIR'] = '/home/lls34/GitHub/01_PhD/PhD_Code/submodules/testing-framework/testing-framework/examples'
os.environ["NSLOTS"]='1'


PSP_PATH = (
    "/home/lls34/programs/QuantumExpresso/pseudopotentials/use_pot"  # Pseudo Potentails
)

# Sceduling Properties
site = SiteConfig("GE", scratchenv="SCRATCH")
site = SiteConfig(scheduler=None)



# Atom specific properties

atoms = atoms

#info = {"uid": atoms.info["uid"], "type": atoms.info["type"]}

prefix = 'test' #"QE_{}_0".format(info["uid"])

j = 0
# Make sure unique output file
while os.path.exists(os.path.join(prefix)) or os.path.exists(
    os.path.join(prefix, ".xyz")
):
    prefix = "QE_{}_{}".format(info["uid"], j)
    j += 1

    if j > 1000:
        raise ValueError("While loop ran too often")
print(f"New prefix {prefix}")

# Now Properites


kspacing = 0.25 / (2.0 * np.pi)

kpts = kspacing_to_grid(atoms, spacing=0.25 / (2.0 * np.pi))
xc = "PBE"
dipole = {"status": False}
calcstress = True
spinpol = True

convergence = {
    "energy": 1e-7,
    "mixing": 0.5,
    "nmix": 10,
    "maxsteps": 500,
    "diag": "david",
}

# if "Slab" in info["type"]:
#     calcstress = False
#     print("type {}: dipole correction on".format(info["type"]))
#     dipole = {"status": True}
#     kpts[2] = 1

print(f"using kpts: {kpts}")

magmom_dict = {"In": 10, "O": 4, "C": 3, "H": 1, "N": 3}
magmoms = []

for i in range(len(atoms)):
    magmoms.append(magmom_dict[atoms[i].symbol])

atoms.set_initial_magnetic_moments(magmoms)

# if "isolated_atom" in info["type"]:
#     calcstress = False
#     spinpol = False
#     kpts = "gamma"
#     atoms.arrays.pop("initial_magmoms")

calc = Espresso(
    # pseudopotentials=pseudopotentials,
    pw=65 * units.Ry,  # plane-wave cut-off in eV, defaults to 350.0
    dw=8 * 65 * units.Ry,  # default 10*pw
    xc=xc,
    kpts=kpts,
    calculation="scf",
    psppath=PSP_PATH,
    parflags="-npool 1",
    nbands=-20,
    occupations="smearing",
    sigma=0.1,
    smearing="gaussian",
    convergence=convergence,
    dipole=dipole,
    calcstress=calcstress,
    spinpol=spinpol,
    # vdw_corr='D3',
    output={
        "removesave": False,
        "avoidio": False,
        "removewf": False,
        "wf_collect": True,
    },
    outdir=prefix,
    site=site,
    txt="pw.out",
)

try:
    atoms.set_calculator(calc)
    print(f"PE: {atoms.get_potential_energy()}")
    #atoms.info = info
    write("{}.xyz".format(prefix), [atoms], format="extxyz")
    atoms = read("{}.xyz".format(prefix))
    #info = atoms.info
    if "stress" in atoms.info.keys():
        info["virial"] = -atoms.info["stress"] / atoms.get_volume()
        atoms.info = info
        print("writing new")
        write("{}.xyz".format(prefix), [atoms], format="extxyz")

except ImportError as e:
    open("{}.FAIL".format(prefix), "w").close()
    print(f"Error: {prefix}, {e}")



TypeError: Invalid initial value for path: None